In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("URINELOTS.xlsx")
data = data[['UHID','RAMJA Results','AIIMS Result culture']]
data.columns = ['UHID','RAMJA','AIIMS']
data.head()

,UHID,RAMJA,AIIMS
0,20220065432,Enterococcus faecalis,Enterococcus faecalis
1,20240016182,NaN,Contamination
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION
4,20201415102,NaN,NaN


In [9]:
df_nan = data[data['RAMJA'].isna()]

In [11]:
df_nan = data[data[['RAMJA', 'AIIMS']].isna().any(axis=1)]
df_nan

,UHID,RAMJA,AIIMS
1,20240016182,NaN,Contamination
4,20201415102,NaN,NaN
7,20240018475,NaN,NaN
20,20240022478,E. coli,NaN
22,20220102808,Sterile,NaN
24,20240017188,E.Faecalis,NaN
30,20240024207,"K pneumoniae, P.mirabilis",NaN
35,20240024631,P aeruginosa,NaN
36,20240022566,"E. Coli, P aeruginosa, P.mirabilis, S aureus, ...",NaN
43,20210126581,"E. Coli, K pneumoniae, A baumannii, E.Faecalis",NaN


In [30]:
#df_nan.to_excel('compareddata.xlsx', sheet_name='Null data')

In [12]:
import re
def cleanup(text):
    text = str(text)
    text = text.upper()
    text = text.replace(".", " ")
    text = text.replace(" AND ", " ")
    text = re.sub(r'\s+', ' ', text)  # Use re.sub for regular expression replacement
    text = text.strip()
    return text


In [13]:
bacteriatypes = {
    "COLI" : 'Escherichia coli', 
    "PNEUMO" : 'Klebsiella pneumoniae',
    "AERUGI" : 'Pseudomonas aeruginosa',
    "BAUMAN" : 'Acinetobacter baumannii', 
    "MIRABIL" : 'Proteus mirabilis', 
    "FAECALI" : 'Enterococcus faecalis', 
    "AUREUS" : 'Staphylococcus aureus', 
    "SAPROPHY" : 'Staphylococcus saprophyticus',
    "STERI" : 'Sterile',
    "CONTA" : 'Contamination',
    "NAN" : 'NAN'
}

In [14]:
data['cleanRAMJA'] = data['RAMJA'].apply(cleanup)
data['cleanAIIMS'] = data['AIIMS'].apply(cleanup)

In [15]:
def check(mainstring, s1, s2):
    if (mainstring.count(s1) > 0 or mainstring.count(s2) > 0):
        #print("YES")
        return(1)
    else:
        #print("NO")
        return(0)

In [16]:
df_R = data.iloc[:,0:6]
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI","CONTA","NAN"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI","CONTA","NAN"]
for s1, s2 in zip(genus,species):
    df_R[s2] = data['cleanRAMJA'].apply(lambda x: check(x, s1, s2))

df_R.rename(columns=bacteriatypes, inplace=True) #change column names
# add suffix _R to column names
# columns_to_modify = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile','Contamination','NAN']
# data.rename(columns={col: f"{col}_R" for col in columns_to_modify}, inplace=True)

In [17]:
df_A = data.iloc[:,0:6]
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI","CONTA","NAN"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI","CONTA","NAN"]
for s1, s2 in zip(genus,species):
    df_A[s2] = data['cleanAIIMS'].apply(lambda x: check(x, s1, s2))

df_A.rename(columns=bacteriatypes, inplace=True) #change column names

# add suffix _R to column names
#columns_to_modify = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile','Contamination','NAN']
#data.rename(columns={col: f"{col}_A" for col in columns_to_modify}, inplace=True)

In [134]:
#data.to_excel("test1.xlsx")

In [18]:
df_R_nonnull = df_R.dropna(subset=['RAMJA','AIIMS'])
df_A_nonnull = df_A.dropna(subset=['RAMJA','AIIMS'])


In [44]:
#...Get infections in Only RAMJA
df_R_m  = df_R.drop(columns= ['UHID','RAMJA', 'AIIMS', 'cleanRAMJA', 'cleanAIIMS','Contamination','Sterile','NAN'])
df_R_m >= 1
df_R_m['Infection_R'] = df_R_m.any(axis=1)
df_R['Infection_R'] = df_R_m['Infection_R'].astype(int)
df_R.head()

,UHID,RAMJA,AIIMS,cleanRAMJA,cleanAIIMS,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile,Contamination,NAN,Infection_R,Infection_A
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0,0,0,1,1
1,20240016182,NaN,Contamination,NAN,CONTAMINATION,0,0,0,0,0,0,0,0,0,0,1,0,0
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,"E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",E COLI,1,1,1,1,0,0,1,0,0,0,0,1,1
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,"K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",CONTAMINATION,0,1,0,1,1,1,1,0,0,0,0,1,0
4,20201415102,NaN,NaN,NAN,NAN,0,0,0,0,0,0,0,0,0,0,1,0,0


In [56]:
df_R_infec = df_R[df_R['Infection_R'] == 1]
df_R_infec = df_R_infec[['UHID','RAMJA','AIIMS']]


In [58]:
df_R_infec.shape

(101, 3)

In [54]:
df_R['Infection_R'].value_counts()

1    101
0     45
Name: Infection_R, dtype: int64

In [60]:
df_A_infec = df_R[df_R['Infection_A'] == 1]
df_A_infec = df_A_infec[['UHID','RAMJA','AIIMS']]
df_A_infec.shape

(30, 3)

In [63]:
df_R['Infection_A'].value_counts()

0    116
1     30
Name: Infection_A, dtype: int64

In [20]:
#...Get infections in Only AIIMS
df_A_m  = df_A.drop(columns= ['UHID','RAMJA', 'AIIMS', 'cleanRAMJA', 'cleanAIIMS','Contamination','Sterile','NAN'])
df_A_m >= 1
df_A_m['Infection_A'] = df_A_m.any(axis=1)
df_A['Infection_A'] = df_A_m['Infection_A'].astype(int)
df_R['Infection_A'] = df_A_m['Infection_A'].astype(int)
df_A.head()

,UHID,RAMJA,AIIMS,cleanRAMJA,cleanAIIMS,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile,Contamination,NAN,Infection_A
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0,0,0,1
1,20240016182,NaN,Contamination,NAN,CONTAMINATION,0,0,0,0,0,0,0,0,0,1,0,0
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,"E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",E COLI,1,0,0,0,0,0,0,0,0,0,0,1
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,"K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",CONTAMINATION,0,0,0,0,0,0,0,0,0,1,0,0
4,20201415102,NaN,NaN,NAN,NAN,0,0,0,0,0,0,0,0,0,0,1,0


In [21]:
df_infection = df_R[['UHID', 'RAMJA', 'AIIMS', 'Infection_A', 'Infection_R']]

In [22]:
#...Get infections in both RAMJA and AIIMS
df_R_nonnull_m  = df_R_nonnull.drop(columns= ['UHID','RAMJA', 'AIIMS', 'cleanRAMJA', 'cleanAIIMS','Contamination','Sterile','NAN'])
df_A_nonnull_m  = df_A_nonnull.drop(columns= ['UHID','RAMJA', 'AIIMS', 'cleanRAMJA', 'cleanAIIMS','Contamination','Sterile','NAN'])
df_C_nonnull = df_A_nonnull_m + df_R_nonnull_m
df_C_nonnull = df_C_nonnull >= 2
df_C_nonnull['Infection_C'] = df_C_nonnull.any(axis=1)
df_R_nonnull['Infection_C'] = df_C_nonnull['Infection_C'].astype(int)
df_R_nonnull.head()

C:\Users\neelesh\AppData\Local\Temp\ipykernel_3740\1918630875.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_R_nonnull['Infection_C'] = df_C_nonnull['Infection_C'].astype(int)


,UHID,RAMJA,AIIMS,cleanRAMJA,cleanAIIMS,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile,Contamination,NAN,Infection_C
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0,0,0,1
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,"E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",E COLI,1,1,1,1,0,0,1,0,0,0,0,1
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,"K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",CONTAMINATION,0,1,0,1,1,1,1,0,0,0,0,0
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...",E.coli,"POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI...",E COLI,1,0,0,1,1,1,1,0,0,0,0,1
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus",E.coli & Pseudomnas,"PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS",E COLI & PSEUDOMNAS,0,0,1,0,1,1,1,0,0,0,0,1


In [29]:
df_concordance = df_R_nonnull[df_C_nonnull['Infection_C'] == 1]
df_concordance = df_concordance[['UHID','RAMJA','AIIMS']]
df_concordance.head()

,UHID,RAMJA,AIIMS
0,20220065432,Enterococcus faecalis,Enterococcus faecalis
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...",E.coli
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus",E.coli & Pseudomnas
8,20230149608,"E. Coli, Pseudomonas, P.mirabilis, E.Faecalis,...","E.coli,A.Baumini"


In [41]:
df_nonnull = data.dropna(subset=['RAMJA','AIIMS'])
df_nonnull = df_nonnull[['UHID','RAMJA','AIIMS']]
df_nonnull.shape

(118, 3)

In [64]:
# Write the DataFrames to an Excel file
with pd.ExcelWriter('compared.xlsx') as writer:
    df_nan.to_excel(writer, sheet_name='Null', index=False)
    df_nonnull.to_excel(writer, sheet_name='Non Null', index=False)
    df_R_infec.to_excel(writer, sheet_name='Ramja Positive', index=False)
    df_A_infec.to_excel(writer, sheet_name='AIIMS Positive', index=False)
    df_concordance.to_excel(writer, sheet_name='Concordance', index=False)

### Confusion matrix bwtween RAMJA and AIIMS

In [84]:
df_infection = df_infection.dropna(subset=['RAMJA','AIIMS'])

In [86]:
df_infection.head()

,UHID,RAMJA,AIIMS,Infection_A,Infection_R
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,1,1
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,1,1
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,0,1
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...",E.coli,1,1
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus",E.coli & Pseudomnas,1,1


In [88]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_infection['Infection_A'],df_infection['Infection_R'])
cm


array([[34, 54],
       [ 7, 23]], dtype=int64)

In [90]:
print(df_infection['Infection_A'].value_counts())
print(df_infection['Infection_R'].value_counts())

0    88
1    30
Name: Infection_A, dtype: int64
1    77
0    41
Name: Infection_R, dtype: int64


In [93]:
df_infection['Infection_R'].sum()

77

In [94]:
df_infection['Infection_A'].sum()

30

In [96]:
df_R_nonnull.shape

(118, 17)

In [95]:
df_R_nonnull['Infection_C'].sum()

17

In [99]:
# df_infection.to_excel('test1.xlsx')
# df_R_nonnull.to_excel('test2.xlsx')

In [91]:
#...Values for RAMJA
N_Ramja  = data.shape[0]
ramjaNull = data['RAMJA'].isnull().sum(axis=0)
ramjanonnull = N_Ramja - ramjaNull
common_nonnull = df_R_nonnull.shape[0]
conta_ramja = df_R['Contamination'].sum()
totalpositive_ramja = df_infection['Infection_R'].sum()

In [92]:
#...Values for AIIMS
N_Aiims  = data.shape[0]
aiimsNull = data['AIIMS'].isnull().sum(axis=0)
aiimsnonnull = N_Aiims - aiimsNull
conta_aiims = df_A['Contamination'].sum()
totalpositive_aiims = df_infection['Infection_A'].sum()

In [20]:
# from sklearn.metrics import confusion_matrix

# y_true = df_A_nonnull['Escherichia coli']
# y_pred = df_R_nonnull['Escherichia coli']

# cm = confusion_matrix(y_true, y_pred)
# print(cm)

[[68 32]
 [ 7 11]]


In [205]:
index = ['TN', 'FP', 'FN', 'TP']
columns = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile']
df_cm = pd.DataFrame(cm_dfs,index,columns)
df_cm

,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile
TN,68,88,103,93,91,88,90,107,42
FP,32,24,13,25,26,24,27,11,12
FN,7,4,0,0,0,3,1,0,38
TP,11,2,2,0,1,3,0,0,26


In [206]:
totalPositive = df_cm.loc['TP'] + df_cm.loc['FP']
totalNegative = df_cm.loc['TN'] + df_cm.loc['FN']
df_cm.loc['Total Positive'] = totalPositive
df_cm.loc['Total Negative'] = totalNegative

In [207]:
df_cm

,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile
TN,68,88,103,93,91,88,90,107,42
FP,32,24,13,25,26,24,27,11,12
FN,7,4,0,0,0,3,1,0,38
TP,11,2,2,0,1,3,0,0,26
Total Positive,43,26,15,25,27,27,27,11,38
Total Negative,75,92,103,93,91,91,91,107,80


In [211]:
aiims2 = [TotalSamples, aiimsNull, aiimsnonnull, common_nonnull, contaaiims]

In [212]:
aiims2

[146, 27, 119, 118, 14]